### Lendo os dados e importando as bibliotecas 

### Criando Distribuição de Frequência por Idade, Raça e Salários

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Supondo que df é o DataFrame com as idades já processadas
# Exemplo de criação de df (caso ainda não tenha sido feito):
# Idades = df['Idade'].astype(str).apply(lambda x: [x[i:i+2] for i in range(0, len(x), 2)]).explode()
# Idades = Idades.astype(int)
# df = pd.DataFrame(Idades, columns=['Idade'])

# Criar a tabela de frequência absoluta
freq_table = df['Idade'].value_counts().sort_index().reset_index()
freq_table.columns = ['Idade', 'Frequency']

# Calcular a frequência relativa (porcentIdadem)
total_pessoas = freq_table['Frequency'].sum()
freq_table['PercentIdade'] = (freq_table['Frequency'] / total_pessoas) * 100

# Exibir a tabela de frequência com porcentIdadem
print("Tabela de Frequência com PorcentIdadem:")
print(freq_table)

# Gerar o gráfico de barras para as frequências absolutas
plt.figure(figsize=(12, 6))
sns.barplot(x='Idade', y='Frequency', data=freq_table, color='blue')
plt.title('Distribuição de Frequência das Idades (Número de Pessoas)')
plt.xlabel('Idade')
plt.ylabel('Número de Pessoas')
plt.xticks(rotation=90)
plt.show()

# Gerar o gráfico de barras para as frequências relativas (porcentIdadem)
plt.figure(figsize=(12, 6))
sns.barplot(x='Idade', y='PercentIdade', data=freq_table, color='green')
plt.title('Distribuição de Frequência das Idades (%)')
plt.xlabel('Idade')
plt.ylabel('Percentual das Idades (%)')
plt.xticks(rotation=90)
plt.show()